In [1]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.cluster as sk
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import random
import torch.optim as optim
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [2]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        hidden = self.relu(hidden)
        output = self.i2o(combined)
        output = self.relu(output)
        output = output.reshape(1)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [3]:
def dataProcess(data):
    data.loc[data.TxGroup == 'Treatment', 'TxGroup'] = 1
    data.loc[data.TxGroup == 'Control', 'TxGroup'] = 0
    #cols_to_norm = ['P1','P2','P3', 'P4', 'P5', 'P6','P7','N1','N2', 'N3', 'N4', 'N5', 'N6', 'N7', 'G1', 'G2', 'G3' , 'G4' , 'G5' , 'G6' , 'G7' , 'G8' , 'G9' , 'G10' , 'G11' , 'G12' , 'G13' , 'G14' , 'G15', 'G16' , 'PANSS_Total']
    #train[cols_to_norm] = preprocessing.scale(train[cols_to_norm])
    return data

In [4]:
#Split the combined studayA-B for training and dev set by creating a list with patient ID 
def splitTrainDev(data_dict):
    #Creare a list by different patient through using patient ID and group by function
    IDkey_list = list(data_dict.groups.keys())
    #Shuffle the patientID in the list
    shuffled_ID = random.sample(IDkey_list, len(IDkey_list))
    #split the training and dev set with 70% training and 30% dev set
    index = round(len(shuffled_ID) * 0.3)
    dev_list = shuffled_ID[index:]
    train_list = shuffled_ID[:index]
    return train_list, dev_list

In [5]:
# randonly select one patient information
def randomPatientSelection(data, key_list):
    #Randomly select one patient
    x_ID = random.choice(key_list)
    #Get the P,N,G features from that ID
    x_patient = data.get_group(x_ID)
    while x_patient.shape[0] == 1:
        x_ID = random.choice(key_list)
        x_patient = data.get_group(x_ID)
    return x_ID, x_patient

In [6]:
# Looping withing one patient information with different visit days
def train_RNN(randSelectedPatient):
    randSelectedPatient = randSelectedPatient[colForTrain]
    hidden = rnn.initHidden()
    optimizer.zero_grad()
    for i in range(randSelectedPatient.shape[0]):
        reshape_randSelectedPatient = torch.tensor(randSelectedPatient.iloc[i].values).float().reshape((1, feature_size))
        output, hidden = rnn(reshape_randSelectedPatient, hidden)
    #Take the last output y to calculate the loss for that randomly selected patient
    y = torch.tensor(randSelectedPatient.PANSS_Total.iloc[-1]).float()
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    return output, loss

In [7]:
# output the prediction for 1 training example
def evaluate(randSelectedPatient):
    randSelectedPatient = randSelectedPatient[colForTrain]
    hidden = rnn.initHidden()
    # print('test 0: ', randSelectedPatient)
    for i in range(randSelectedPatient.shape[0]):
        reshape_randSelectedPatient = torch.tensor(randSelectedPatient.iloc[i].values).float().reshape((1, feature_size))
        output, hidden = rnn(reshape_randSelectedPatient, hidden)
        # print('test 1')
    y = torch.tensor(randSelectedPatient.PANSS_Total.iloc[-1]).float()
    # print('test 2')
    loss = criterion(output, y)
    return output, loss


In [8]:
# Define evaluation matric
def calculate_RMSE(groupbyObject, key_list):
    #The patient number in the list 
    m = len(key_list)
    outputs = torch.zeros((m, 1))
    RMSEs = torch.zeros((m, 1))
    # loop through all the patient and their last PANSS score and calculare the RMSE, and then average all the RMSE
    for i in range(m):
        output, loss = evaluate(groupbyObject.get_group(key_list[i]))
        outputs[i,:] = output
        RMSEs[i, :] = torch.sqrt(loss)
    avg_RMSEs = torch.mean(RMSEs).item()
    return avg_RMSEs, outputs


In [9]:
train_path = 'c:/stats202/Objective3_visitdaydifferencedata.csv'
test_path = 'c:/stats202/Objective3_df_test.csv'
train = pd.read_csv(train_path, header= 0)
test = pd.read_csv(test_path, header= 0)
train.shape
train

,Study,Country,PatientID,SiteID,RaterID,AssessmentiD,TxGroup,VisitDay,P1,P2,...,G9,G10,G11,G12,G13,G14,G15,G16,PANSS_Total,LeadStatus
0,A,USA,10001,20035,30076,100679,Control,0,5,5,...,5,3,3,4,3,3,3,5,107,Assign to CS
1,A,USA,10001,20035,30076,101017,Control,11,5,5,...,5,3,3,4,3,3,3,5,109,Assign to CS
2,A,USA,10001,20035,30076,102177,Control,7,4,4,...,4,2,2,3,3,2,3,4,91,Passed
3,A,USA,10001,20035,30076,101533,Control,7,3,3,...,3,2,2,3,3,2,3,4,80,Flagged
4,A,USA,10001,20035,30076,100930,Control,14,3,3,...,3,2,2,3,3,2,3,4,77,Flagged
5,A,USA,10001,20035,30076,100471,Control,14,3,3,...,3,2,2,3,3,2,3,4,75,Flagged
6,A,USA,10001,20035,30076,102347,Control,14,4,2,...,3,2,2,3,3,2,3,4,72,Flagged
7,A,USA,10002,20011,30016,100597,Control,0,5,5,...,5,2,1,3,3,3,3,5,85,Passed
8,A,USA,10002,20011,30016,100270,Control,7,5,5,...,5,3,1,3,3,1,3,5,85,Passed
9,A,USA,10002,20011,30016,101211,Control,2,5,5,...,5,3,1,3,3,1,3,5,94,Passed


In [10]:
colForTrain = ['TxGroup','VisitDay','P1','P2','P3', 'P4', 'P5', 'P6','P7','N1','N2', 'N3', 'N4', 'N5', 'N6', 'N7', 'G1', 'G2', 'G3' , 'G4' , 'G5' , 'G6' , 'G7' , 'G8' , 'G9' , 'G10' , 'G11' , 'G12' , 'G13' , 'G14' , 'G15', 'G16', 'PANSS_Total']
input_col = ['PatientID', 'TxGroup','VisitDay','P1','P2','P3', 'P4', 'P5', 'P6','P7','N1','N2', 'N3', 'N4', 'N5', 'N6', 'N7', 'G1', 'G2', 'G3' , 'G4' , 'G5' , 'G6' , 'G7' , 'G8' , 'G9' , 'G10' , 'G11' , 'G12' , 'G13' , 'G14' , 'G15', 'G16','PANSS_Total']

feature_size = len(input_col) - 1
train = dataProcess(train)[input_col]
test = dataProcess(test)[input_col]


train_dict = train.groupby('PatientID')
train_PatientID_list, dev_PatientID_list= splitTrainDev(train_dict)
test_dict = test.groupby('PatientID')
test_PatientID_list = list(test_dict.groups.keys())

print('test_0')
# define the demision of the parameters of RNN
n_hidden = 5
n_features = 33
n_categories = 1
# define the the parameters for plotting
n_iters = 2500
print_every = 50
plot_every = 50

# Keep track of losses for plotting
all_losses_diffLR = {}

# learning_rate = [0.5, 0.05, 0.005, 0.0005, 0.00005]
learning_rate = [0.0005]


test_0


In [11]:
for i in range(len(learning_rate)):
    train_current_loss = 0
    train_all_losses = []

    rnn = RNN(n_features, n_hidden, n_categories)
    # Define the cost function
    criterion = nn.MSELoss()
    # Define the optimizer
    optimizer = optim.Adam(rnn.parameters(), lr=learning_rate[i])

    for iter in range(1, n_iters + 1):
        #Randomly select one patient information
        x_ID, x_patient = randomPatientSelection(train_dict, train_PatientID_list)
        #Print out the output and loss for that patient with different visit days
        train_output, train_loss = train_RNN(x_patient)
        train_loss = train_loss.item()
        train_current_loss += train_loss  # Add current loss avg to list of losses

        if iter % print_every == 0:
            print('current training loss at ' + str(iter) + '-th iteration: ' + str(train_current_loss))

        if iter % plot_every == 0:
            train_all_losses.append(train_current_loss / plot_every)
            train_current_loss = 0

    all_losses_diffLR[learning_rate[i]] = train_all_losses
    
    np.savetxt('c:/stats202/objective3/feature = all score_visit_diff/learning_rate = 0.0005_num_2500_no_normalize_2.csv', all_losses_diffLR[learning_rate[i]], delimiter=',')

    training_RMSE, train_pred = calculate_RMSE(train_dict, train_PatientID_list)
    dev_RMSE, dev_pred = calculate_RMSE(train_dict, dev_PatientID_list)
    test_RMSE, test_pred = calculate_RMSE(test_dict, test_PatientID_list)

    
    avg_training_RMSE = np.mean(training_RMSE)
    avg_dev_RMSE = np.mean(dev_RMSE)
    avg_test_RMSE = np.mean(test_RMSE)
    
    np.savetxt('c:/stats202/objective3/feature = all score_visit_diff/learning_rate = 0.0005_num_2500_test_predno_normalize_2.csv', test_pred.detach().numpy(), delimiter=',')

    print('training RMSE with lr =  ' + str(learning_rate[i]) + ': ' + str(avg_training_RMSE))
    print('dev RMSE with lr = ' + str(learning_rate[i]) + ': ' + str(avg_dev_RMSE))
    print('test RMSE with lr = ' + str(learning_rate[i]) + ': ' + str(avg_test_RMSE))
    #print('test RMSE with lr = ' + str(learning_rate[i]) + ': ' + str(test_pred))

current training loss at 50-th iteration: 174742.78411865234
current training loss at 100-th iteration: 143222.3875732422
current training loss at 150-th iteration: 94409.5200805664
current training loss at 200-th iteration: 62744.478702545166
current training loss at 250-th iteration: 33534.36951160431
current training loss at 300-th iteration: 12621.243667900562
current training loss at 350-th iteration: 10701.413489304483
current training loss at 400-th iteration: 5510.981344456086
current training loss at 450-th iteration: 6905.117378659546
current training loss at 500-th iteration: 10717.445327888243
current training loss at 550-th iteration: 4375.233742500655
current training loss at 600-th iteration: 2352.400581534952
current training loss at 650-th iteration: 3013.352400075062
current training loss at 700-th iteration: 6246.853201970458
current training loss at 750-th iteration: 6390.3514250814915
current training loss at 800-th iteration: 3490.768982309848
current training los